## Check List 3 - SceneBuilder, GeometryHandle
* **3.1 SceneBuilder**  
  - reset_reference_coord, detect_items, create_gscene, add_robot_geometries, add_poles, detect_and_register
* **3.2 GeometryHandle**  
  - create_safe
    - BOX, CYLINDER, PLANE, CAPSULE, SPHERE, ARROW
  - show_pose, show_motion

## set running directory to project source

In [1]:
import os
import sys
import numpy as np
import time
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 3.1. SceneBuilder

##### Create CombinedRobot instance - you need it to initailize GeometryScene

In [2]:
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

combined_robot = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))], 
    connection_list=[False, False])

connection command:
indy0: False
panda1: False


##### initialize ArucoStereo detector

In [3]:
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[Kinect(), RealSense()])

stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()
print("Detector initialized")

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming
Detector initialized


##### Create SceneBuilder instance

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
scene_builder = SceneBuilder.instance(detector=stereo)

##### detect_items()
* Detect items in the field of view. Here, we detect robot positions and update it to the combined robot

In [6]:
xyz_rpy_robots = scene_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
combined_robot.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)

##### create_gscene()
* Create a GeometryScene instance

In [7]:
gscene = scene_builder.create_gscene(combined_robot)

ros_node already initialized somewhere else
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


##### add_robot_geometries()
* Add collision boundaries defined in the URDF/Xacro file of each robots

In [8]:
gtems = scene_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

Please create a subscriber to the marker


##### add_poles()
* This adds poles from reference plane to given position. This is mainly used to make collision boundaries for camera poles

In [11]:
ptems = scene_builder.add_poles(
    {"cam0": stereo.ref_coord_inv[:3,3], 
     "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))

##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ

In [12]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

## 3.2. GeometryScene
* GeometryScene deals with GeometryItem instances and RVIZ visualization. It is created by SceneBuilder above.

##### create_safe()
* Creates new GeometryItem in the scene
* BOX and CYLINDER are good with Moveit
* PLANE can be used for generating motion constaint in Moveit
* SPHERE and CAPSULE are not supported in Moveit (SPHERE is not applied correctly)
* CYLINDER is converted to CAPSULE in eTaSL (CYLINDER is not applied correctly)
* ARROW is used only for axis visualization

In [13]:
from pkg.geometry.geotype import GEOTYPE

# generate box geometry
gscene.create_safe(name="test_box", link_name="base_link", gtype=GEOTYPE.BOX,
                                center=(-0.5,0.0,0.05), rpy=(0,0,0), dims=(0.1,0.1,0.1),
                                color=(1,0,0,1), display=True, collision=True, fixed=True)

# generate cylinder geometry
gscene.create_safe(name="test_cyl", link_name="base_link", gtype=GEOTYPE.CYLINDER,
                                center=(-0.3,0.0,0.1), rpy=(0,0,0), dims=(0.1,0.1,0.2),
                                color=(0,1,0,1), display=True, collision=True, fixed=True)

# generate plane geometry
gscene.create_safe(name="test_plane", link_name="base_link", gtype=GEOTYPE.PLANE,
                                center=(-0.1,0.0,0.05), rpy=(np.pi/2,0,0), dims=(0.1,0.1,0.001),
                                color=(0,0,1,1), display=True, collision=True, fixed=True)

# generate sphere geometry
gscene.create_safe(name="test_sphere", link_name="base_link", gtype=GEOTYPE.SPHERE,
                                center=(0.1,0.0,0.05), rpy=(0,0,0), dims=(0.1,0.1,0.1),
                                color=(1,1,0,1), display=True, collision=True, fixed=True)

# generate capsule geometry
gscene.create_safe(name="test_capsule", link_name="base_link", gtype=GEOTYPE.CAPSULE,
                                center=(0.3,0.0,0.1), rpy=(0,0,0), dims=(0.1,0.1,0.1),
                                color=(0,1,1,1), display=True, collision=True, fixed=True)

# generate arrow geometry
gscene.create_safe(name="test_arrow", link_name="base_link", gtype=GEOTYPE.ARROW,
                                center=(0.5,0.0,0.05), rpy=(0,0,0), dims=(0.1,0.02,0.02),
                                color=(1,0,1,1), display=True, collision=True, fixed=True)

##### show_pose()
* show pose in RVIZ

In [14]:
gscene.show_pose(combined_robot.home_pose)

##### show_motion()
* show motion in RVIZ

In [15]:
N_div = 100
trajectory = [combined_robot.home_pose + 0.2 * (np.sin(2*np.pi * (float(i) / N_div))) / 2 for i in range(N_div)]
gscene.show_motion(trajectory, period=0.01)

## Finished
#### Stop and restart the kernel for next test